In [1]:
import black
import jupyter_black

jupyter_black.load(
    lab=True,
    line_length=110,
    target_version=black.TargetVersion.PY310,
)

In [2]:
import os
import subprocess

import GooseSLURM as gs
import numpy as np

import lysis

In [3]:
scenario_type = np.dtype(
    [
        ("descriptor", np.str_, 40),
        ("total_molecules", int),
        ("pore_size", float),  # in microns. Code requires cm
        ("fiber_diameter", float),  # in microns. Needs to be added to pore_size to get grid_node_distance
        ("cols", int),
        ("rows", int),
        ("forced_unbind", float),
    ]
)
mechanism_type = np.dtype([("descriptor", np.str_, 40), ("executable", np.str_, 40)])
run_type = np.dtype(
    [
        ("exp_code", np.str_, 15),
        ("scenario", np.str_, 40),
        ("mechanism", np.str_, 40),
        ("seed", int),
        ("running_time", int),
    ]
)
diameter_code = {
    72.7: "Q2",
    145.4: "Q4",
}

In [4]:
# seed sequence entropy: 3881821051554698152964433817123076384
scenarios = np.array(
    [
        ("TN-L_9350", 9_350, 1.0135, 72.7, 93, 121, 0.0852),
        ("TN-L_307", 307, 1.0135, 72.7, 93, 121, 0.0852),
        ("TN-D_684", 684, 0.22, 72.7, 342, 144, 0.0852),
        ("TN-D_9350", 9_350, 0.22, 72.7, 342, 144, 0.0852),
        ("TN-D_307", 307, 0.22, 72.7, 342, 144, 0.0852),
        ("TK-L_3042", 3_042, 1.0135, 145.4, 87, 67, 0.0729129),
        ("TK-L_9350", 9_350, 1.0135, 145.4, 87, 67, 0.0729129),
        ("TK-L_307", 307, 1.0135, 145.4, 87, 67, 0.0729129),
        ("TK-D_307", 307, 0.22, 145.4, 274, 65, 0.0729129),
        ("TK-D_9350", 9350, 0.22, 145.4, 274, 65, 0.0729129),
    ],
    dtype=scenario_type,
)
mechanisms = np.array(
    [
        ("Into and along - Internal", "macro_diffuse_into_and_along__internal"),
    ],
    dtype=mechanism_type,
)

In [5]:
in_file_code = "_PLG2_tPA01_{input_code}.dat"
out_file_code = "_{scenario_code}.dat"

In [6]:
group_code = "2023-04-17-14"
runs = np.array(
    [
        # (group_code + "00", "TN-L_9350", "Into and along - Internal", 1_034_836_197, 0),
        # (group_code + "01", "TN-L_307", "Into and along - Internal", 2_464_079_704, 0),
        # (group_code + "02", "TN-D_684", "Into and along - Internal", 982_919_027, 1800),
        # (group_code + "03", "TN-D_9350", "Into and along - Internal", 129_314_213, 1800),
        # (group_code + "04", "TN-D_307", "Into and along - Internal", 2_705_873_059, 1800),
        # (group_code + "05", "TK-L_3042", "Into and along - Internal", 2_104_158_012, 0),
        # (group_code + "06", "TK-L_9350", "Into and along - Internal", 2_608_028_918, 0),
        # (group_code + "07", "TK-L_307", "Into and along - Internal", 1_819_293_232, 0),
        # (group_code + "08", "TK-D_307", "Into and along - Internal", 3_128_495_922, 1800),
        (group_code + "09", "TK-D_9350", "Into and along - Internal", 106_672_317, 1800),
    ],
    dtype=run_type,
)

In [7]:
for run in runs:
    mech = mechanisms[mechanisms["descriptor"] == run["mechanism"]][0]
    scen = scenarios[scenarios["descriptor"] == run["scenario"]][0]
    e = lysis.util.Experiment(
        os.path.join("/", "home", "bpaynter", "git", "UCO-OpResearch", "lysis", "data"),
        experiment_code=run["exp_code"],
    )
    p = {
        "total_molecules": int(scen["total_molecules"]),
        "pore_size": scen["pore_size"] / 10_000,  # convert to cm
        "grid_node_distance": scen["fiber_diameter"] / 1_000 + scen["pore_size"],
        "cols": int(scen["cols"]),
        "rows": int(scen["rows"]),
        "forced_unbind": scen["forced_unbind"],
        "seed": int(np.int32(run["seed"])),
        "total_time": int(run["running_time"]),
        "macro_version": "macro_diffuse_into_and_along__internal",
    }
    e.initialize_macro_param(p)
    e.to_file()

    with open(os.path.join(e.os_path, "README.md"), "w") as file:
        file.write(
            f""" ## {e.experiment_code}
Internal lysis experiments

Uses Fortran Macroscale code with
    -  Diffuse into and along
    -  'restricted move' bug correction
    -  'passerby molecule' bug correction
    -  addition of extra output data
    -  changing macro-unbind wait time to remaining leaving time.
    -  changing the molecule start location calculation
    -  eliminating unused data variables

These sets are 30 min simulations
    
This set of data is from the "{run['scenario']}" set.
"""
        )

    result = subprocess.run(
        ["cp", os.path.join("src", "fortran", mech["executable"] + ".f90"), e.os_path],
        cwd=os.path.join("/", "home", "bpaynter", "git", "UCO-OpResearch", "lysis"),
        capture_output=True,
    )
    print(result)

CompletedProcess(args=['cp', 'src/fortran/macro_diffuse_into_and_along__internal.f90', '/home/bpaynter/git/UCO-OpResearch/lysis/data/2023-04-17-1409'], returncode=0, stdout=b'', stderr=b'')


In [8]:
for run in runs:
    mech = mechanisms[mechanisms["descriptor"] == run["mechanism"]][0]
    scen = scenarios[scenarios["descriptor"] == run["scenario"]][0]
    e = lysis.util.Experiment(
        os.path.join("/", "home", "bpaynter", "git", "UCO-OpResearch", "lysis", "data"),
        experiment_code=run["exp_code"],
    )
    e.read_file()
    f = lysis.util.FortranMacro(
        exp=e,
        executable=os.path.join(
            "/", "home", "bpaynter", "git", "UCO-OpResearch", "lysis", "bin", mech["executable"]
        ),
        in_file_code=in_file_code.format(input_code=diameter_code[scen["fiber_diameter"]]),
        out_file_code=out_file_code.format(scenario_code=run["scenario"]),
    )
    comm = f.exec_command()
    comm += [
        ">",
        os.path.join(
            e.os_path,
            "macro" + out_file_code[:-4].format(scenario_code=run["scenario"]) + ".txt",
        ),
    ]
    comm = " ".join(comm)
    # job-options
    sbatch = {
        "job-name": "lysis-" + e.experiment_code,
        "out": os.path.join(e.os_path, "job.slurm-%j.out"),
        "nodes": 1,
        "mem": 3096,
        "ntasks": 1,
        "cpus-per-task": 1,
        "partition": "long",
    }
    script = ["module purge", "module load oneapi/compiler", comm]
    with open(os.path.join(e.os_path, "job.slurm"), "w") as file:
        file.write(gs.scripts.plain(script, **sbatch))
    result = subprocess.run(
        ["sbatch", os.path.join(e.os_path, "job.slurm")],
        cwd=os.path.join("/", "home", "bpaynter", "git", "UCO-OpResearch", "lysis"),
        capture_output=True,
    )
    print(result)

CompletedProcess(args=['sbatch', '/home/bpaynter/git/UCO-OpResearch/lysis/data/2023-04-17-1409/job.slurm'], returncode=0, stdout=b'Submitted batch job 2215992\n', stderr=b'')
